# MSU Curriculum Core API - Demo

## 1. Import the API

In [ ]:
import curriculum_core as api

## 2. Load Data

In [ ]:
api.load_data(
    registrar_path="20250919_Registrars_Data(in).csv",
    majors_path="CNS_Majors_Data.xlsx"
)

## 3. Get Statistics

In [ ]:
stats = api.get_statistics()
print(f"Total courses: {stats['total_courses']}")
print(f"Courses with prerequisites: {stats['courses_with_prereqs']}")
print(f"Total subjects: {stats['total_subjects']}")
print(f"Max prerequisite depth: {stats['max_prereq_depth']}")

## 4. Get Course Information
### Returns dict with separate subject/number

In [ ]:
course = api.get_course("CSE 232")
print(f"Course: CSE 232")
print(f"Name: {course['name']}")
print(f"Subject: {course['subject']}")
print(f"Number: {course['number']} (type: {type(course['number']).__name__})")
print(f"\nFull dict: {course}")

## 5. Get Prerequisites
### Returns {"CSE 231": {...}, "MTH 132": {...}}

In [ ]:
prereqs = api.get_prerequisites("CSE 232")

print(f"CSE 232 has {len(prereqs)} prerequisite courses:\n")
for prereq_id, data in prereqs.items():
    print(f"  {prereq_id}:")
    print(f"    Name: {data['name']}")
    print(f"    Subject: {data['subject']}, Number: {data['number']}")

# Get detailed prerequisite info with AND/OR logic
print("\n" + "="*70)
print("Detailed prerequisite information:")
details = api.get_prerequisite_details("CSE 232")
print(f"Formatted: {details['formatted']}")
print(f"Depth: {details['depth']}")
print(f"Total count: {details['total_count']}")
print(f"Groups: {details['prerequisite_groups']}")

## 6. Get Dependent Courses
### Uses adjacency list for fast lookup

In [ ]:
dependents = api.get_dependent_courses("MTH 132")

print(f"MTH 132 is required by {len(dependents)} courses:\n")
print("First 10 courses:")
for course_id, data in list(dependents.items())[:10]:
    print(f"  {course_id}: {data['name']}")

if len(dependents) > 10:
    print(f"  ... and {len(dependents) - 10} more")

## 7. Search for Courses

In [ ]:
results = api.search_courses("calculus", limit=10)

print(f"Found {len(results)} calculus courses:\n")
for course_id, data in results.items():
    print(f"  {course_id}: {data['name']}")
    print(f"    Subject: {data['subject']}, Number: {data['number']}")

## 8. Get All CSE Courses
### Course ID is the key

In [ ]:
cse_courses = api.get_courses_by_subject("CSE")

print(f"Total CSE courses: {len(cse_courses)}\n")
print("First 10 CSE courses:")
for course_id, data in list(cse_courses.items())[:10]:
    print(f"  {course_id}:")
    print(f"    Name: {data['name']}")
    print(f"    Number: {data['number']} (type: {type(data['number']).__name__})")

# TEST: Sort by number (now works because number is int!)
print("\n" + "="*70)
print("Sorted by course number:")
sorted_courses = sorted(cse_courses.items(), key=lambda x: x[1]['number'])
for course_id, data in sorted_courses[:10]:
    print(f"  {course_id} (number={data['number']})")

## 9. Detect Prerequisite Cycles

In [ ]:
cycles = api.detect_cycles()

print(f"Prerequisite cycle detection:")
if cycles:
    print(f"WARNING: Found {len(cycles)} cycles!\n")
    print("First 5 cycles:")
    for i, cycle in enumerate(cycles[:5], 1):
        print(f"  {i}. {' -> '.join(cycle)} -> {cycle[0]}")
else:
    print("No cycles detected - prerequisite structure is valid!")

## 10. Bottleneck Analysis
### Find courses that block the most other courses

In [ ]:
bottlenecks = api.get_bottleneck_courses(major="7105", top_n=10)

print("Top 10 Bottleneck Courses for Chemistry Major (7105):")
print("(These should be taken EARLY to avoid blocking other courses)\n")

for i, (course_id, data) in enumerate(bottlenecks.items(), 1):
    print(f"{i}. {course_id}: blocks {data['blocks']} courses")
    print(f"   Required by: {', '.join(data['dependent_courses'][:5])}...")
    print()

## 11. Get Major List

In [ ]:
majors = api.get_major_list()

print(f"Total majors: {len(majors)}\n")
print("First 10 majors:")
for major_code, data in list(majors.items())[:10]:
    print(f"  {major_code}: {data['name']} ({data['courses']} courses)")

## 12. Test Data Structure Consistency

In [ ]:
print("Testing data structure consistency:\n")

# Test 1: All functions return dicts
all_courses = api.get_all_courses()
search_results = api.search_courses("CSE", limit=5)
cse_courses = api.get_courses_by_subject("CSE")
prereqs = api.get_prerequisites("CSE 232")
dependents = api.get_dependent_courses("CSE 231")

tests = [
    ("get_all_courses()", all_courses),
    ("search_courses()", search_results),
    ("get_courses_by_subject()", cse_courses),
    ("get_prerequisites()", prereqs),
    ("get_dependent_courses()", dependents)
]

for func_name, data in tests:
    is_dict = isinstance(data, dict)
    status = "✓" if is_dict else "✗"
    print(f"{status} {func_name}: {type(data).__name__} with {len(data)} items")
    
    # Check structure of first item
    if data and is_dict:
        first_key = list(data.keys())[0]
        first_value = data[first_key]
        if isinstance(first_value, dict):
            has_required = all(field in first_value for field in ['name', 'subject', 'number'])
            number_is_int = isinstance(first_value.get('number'), int)
            print(f"     └─ Structure OK: {has_required}, Number is int: {number_is_int}")

print("\n✓ All functions return consistent dict-of-dicts format!")

## 13. Demonstrate Sorting by Number

In [ ]:
cmse_courses = api.get_courses_by_subject("CMSE")

print("CMSE Courses sorted by number (ascending):")
sorted_courses = sorted(cmse_courses.items(), key=lambda x: x[1]['number'])

for course_id, data in sorted_courses:
    print(f"  {course_id:12} (number={data['number']:3}) {data['name'][:50]}")

print("\n Numeric sorting works correctly!")
print("   (Note: 201 < 314 < 401 < 801, not '201' < '314' < '401' < '801')")

## 14. Graph Data

In [ ]:
graph_data = api.get_graph_data(major="7105", max_depth=2)

print(f"Graph data for Chemistry Major (7105):")
print(f"  Nodes: {len(graph_data['nodes'])} (dict-of-dicts)")
print(f"  Edges: {len(graph_data['edges'])}")

print("\nSample nodes (course_id is key):")
for course_id, node_data in list(graph_data['nodes'].items())[:5]:
    print(f"  {course_id}:")
    print(f"    Name: {node_data['name']}")
    print(f"    Year: {node_data['year']}, Depth: {node_data['depth']}")

## 15. Graph Visualization Demo
### Create and visualize the Chemistry Major prerequisite graph

In [ ]:
# Get graph data
graph_data = api.get_graph_data(major="7105", max_depth=2)
print(f"Graph has {len(graph_data['nodes'])} courses")

In [ ]:
# Visualize it
api.visualize_graph(
    graph_data,
    layout="spring",
    color_by="depth", # you can change color from depth to year
    figsize=(20, 15),
    title="Chemistry Major - Prerequisite Flow",
    save_path="chem_graph.png"
)

In [ ]:
# Analyze it
api.print_graph_analysis(graph_data, top_n=5)

## 16. Quick Help

In [ ]:
api.help_quick()